In [22]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.tseries.offsets import BDay
from hsmm_core.hmm import hmm_engine
from hsmm_core.observation_models import ExpIndMixDiracGauss
from hsmm_core.data_utils import load_data, TradingHours
from hsmm_core.data_utils import load_data, TradingHours
from hsmm_core.feature_spaces import hmm_features
from hsmm_core.hmm import hmm_calibration
from hsmm_core.data_utils import load_data, TradingHours
from hsmm_core.labelling import DataLabellingSimple
from hsmm_core.consts import ThresholdMethod, LabellingChoice
import pickle
from hsmm_core.consts import InitialisationMethod
import datetime as dt
plt.style.use('ggplot')
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier,  GradientBoostingClassifier
from sklearn.linear_model import RidgeClassifierCV
sc = StandardScaler()

In [23]:
def remove_nans(features_tuple, labels, idx=1):
    # not the cleanest but useful
    # function to clean up nans as I seem to use it a lot, so better to have one function
    # combines the features and labels and removes rows with nans across so we dont lose the ordering
    # returns features and labels
    features_df = pd.concat([features_tuple[0], features_tuple[1], features_tuple[2], \
                             features_tuple[3]], axis=1, sort=False)
    labels_only = labels.drop(columns=['ReturnTradedPrice', 'Duration', 'states', 'TradedTime',
                                       'TradedPrice', 'ticker'], axis=1)
    df_concat = pd.concat([features_df, labels_only.iloc[:, 0:idx]], axis=1, sort='False')
    # only using 1st set of labels- but we can re-write this a bit
    df_x_nan = df_concat.dropna()  # dropping all nans
    label_column_loc_ = df_x_nan.shape[1] - 1  # location of labels column in the clean df
    labels_ = df_x_nan.iloc[:, label_column_loc_:label_column_loc_ + 1]  # keep pure labels
    features_ = df_x_nan.drop(df_x_nan.columns[label_column_loc_], axis=1)  # keeping the features only

    return features_, labels_ #return features and labels in the X,y order that scikit takes the input


In [24]:
class FitModels(object):
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    # # Train a SVM classification model

    def ridge_clf(self, cv_folds=5):
        model_ridge_clf = RidgeClassifierCV(alphas=np.arange(0.1, 1000, 0.1), \
                                            cv=KFold(cv_folds), normalize=True).fit(self.X_train,
                                                                                    self.y_train.values.ravel())
        # check if class_weight should be used as 'balanced'

        return model_ridge_clf

    def svm_clf(self, kernel_choice):
        param_grid = dict(kernel=[str(kernel_choice)],
                          C=[1, 5, 10, 25, 50, 100],
                          gamma=[0.0001, 0.001, 0.01, 0.02, 0.05, 0.01])
        svc = SVC(class_weight='balanced')
        clf = GridSearchCV(svc, param_grid)
        clf.fit(self.X_train, np.asanyarray(self.y_train).reshape(self.y_train.shape[0]))

        return clf

    def gradient_boost_clf(self, learning_rate=0.25):
        # this needs to be written properly- but this is somewhat optimised#
        GBR = GradientBoostingClassifier(n_estimators=3000, learning_rate=learning_rate,
                                         max_depth=4, max_features='sqrt',
                                         min_samples_leaf=15, min_samples_split=10)

        gb_boost_clf = GBR.fit(self.X_train, self.y_train)

        return gb_boost_clf

    def gp_clf(self):
        kernel = 1.0 * RBF([1.0])  # isotropic
        gpc_rbf_isotropic = GaussianProcessClassifier(kernel=kernel).fit(self.X_train, self.y_train)
        # hyperparameters are optimised by default
        return gpc_rbf_isotropic

    def random_forest_clf(self, no_est=100):
        rfc = RandomForestClassifier(n_estimators=no_est, max_depth=4, n_jobs=-1, warm_start=True)
        rfc.fit(X_train, y_train)

        return rfc

    def run_cv(self, clf_class, **kwargs):
        # Construct a kfolds object
        kf = KFold(len(self.y_train), n_folds=10, shuffle=True)
        y_pred = self.y_train.copy()

        # Iterate through folds
        for train_index, test_index in kf:
            X_train_local, X_test_local = self.X_train[train_index], self.X_train[test_index]
            y_train_local = self.y_train[train_index]
            # Initialize a classifier with key word arguments
            clf = clf_class(**kwargs)
            clf.fit(self.X_train, self.y_train)
            y_pred[test_index] = clf.predict(X_test_local)
        return y_pred

In [25]:

ticker = 'test_SYNT_2states'


data_dir = os.getenv('FINANCE_DATA')
features_path='/home/ak/Data/features_models/features/'
labels_path= '/home/ak/Data/features_models/labels'

ticker_labels_path = os.path.join(labels_path,ticker+'/NON_DIRECTIONAL')

if not os.path.exists(os.path.join(data_dir, ticker)):
    os.makedirs(os.path.join(data_dir, ticker))
    
if not os.path.exists(ticker_labels_path):
    os.makedirs(ticker_labels_path)

    ####paths####
main_path = '/home/ak/Data/features_models/'

models_path=os.path.join(main_path,'models')
ticker_models_path = os.path.join(models_path, ticker)
# hmm_models_path = os.path.join(models_path,'hmm_models')
# features_ticker_path = os.path.join(features_path, ticker)
# predictions_path = os.path.join(main_path, 'predictions')
if not os.path.exists(ticker_models_path):
    os.makedirs(ticker_models_path)


In [26]:
ticker_labels_path

'/home/ak/Data/features_models/labels/test_SYNT_2states/NON_DIRECTIONAL'

In [5]:

no_states = 2
sigmas = [0.05, 0.002] # fast and slow
# Duration is measured in seconds for now (to be revised). lambda units are seconds^{-1}
# so here we consider

lambdas = [1./35., 1./10.]
weights = [0.1, 0.6]

obs_model = ExpIndMixDiracGauss(no_states)
obs_model.set_up_initials(priors={'sigmas': sigmas, 'lambdas': lambdas, 'weights': weights})

hmm_ = hmm_engine(obs_model, no_states)

# set up some priors
tpm = np.array([[0.4, 0.6], [0.7, 0.3]])
pi = np.array([0.4, 0.6])
hmm_.set_up_initials(priors={'tpm': tpm, 'pi': pi})

no_dates = 3
start_date = pd.datetime(2017, 6, 1)
dummy_dates = [start_date + BDay(i) for i in range(no_dates)]

no_points = 5000

rng = np.random.RandomState(1234)



In [6]:

# silly hack, add 1 millisecond so that the initial timestamp is printed with milliseconds and does not
# break the parsing of Timestamps when loading

morning_start = dt.time(8, 0, 0, 1)

initial_price = 100

for dd in dummy_dates:
    random_states = hmm_.sample_states(rng=rng, length=no_points)
    observation_points = obs_model.sample_data(no_points, rng=rng, state=random_states)
    # The first duration is always zero
    observation_points[0, 0] = 0.

    file_path = os.path.join(data_dir, ticker)
    file_name = '.'.join([dd.strftime('%Y%m%d'), 'csv'])

    data_to_save = pd.DataFrame({'states': random_states,
                                 'Duration': observation_points[:, 0],
                                 'ReturnTradedPrice': observation_points[:, 1],
                                 })
    data_to_save['TradedTime'] = pd.Series()

    # Now calculate the Traded prices and traded times in reverse order as to what would happen
    # with real data.
    # data_to_save.loc[0, 'TradedTime'] = dt.datetime.combine(dd.date(), morning_start)
    data_to_save['TradedTime'] = data_to_save['Duration'].cumsum().apply(lambda dur:
                                                                         (dt.datetime.combine(dd.date(), morning_start)+\
                                                                                     dt.timedelta(seconds=dur)).time())

    data_to_save['TradedPrice'] = initial_price * (1. + data_to_save['ReturnTradedPrice']).cumprod()
    data_to_save.to_csv(os.path.join(file_path, file_name), index=False)

print "ok"


ok


### Feature Creation ###

In [7]:

n_hidden_states = no_states

init_params = {
    "obs_model_params": {
                                'obs_model_name': 'ExpIndMixDiracGauss',
                                'em_init_method': InitialisationMethod.cluster

    },
    "hidden_model_params": {
                                'no_hidden_states': no_states,
                                'pi':pi,
                                'tpm': tpm,
                                'em_init_method': InitialisationMethod.uniform
    },
    "update_tag": 'tpsml'
}


data = load_data(ticker, which_trading_hours=TradingHours.all_trading_day)

In [8]:
trd_hours_filter = TradingHours.all_trading_day
hmm_calibration_engine = hmm_calibration(no_parallel_procs=None,
                                         init_params=init_params)


trained_hmms = hmm_calibration_engine.hmm_fit_func(ticker, data, trd_hours_filter,
                                                   force_recalc=False)


for date, date_hmm in trained_hmms.iteritems():
    feature_engine = hmm_features(date_hmm)
    features = feature_engine.generate_features(data[date])

## uncomment below:  for saving hmm models ##

In [ ]:
# ###saving trained model hmms###
# seq_model = "_".join((str(ticker),str(n_hidden_states),'state',"trained","hmm","models", ".pickle"))
# print("saving the model:", seq_model)
# pickle.dump(init_params, open(os.path.join(models_path,seq_model), 'wb'))

## create labels ##

In [17]:
window =25
threshold =0.1


labelling_method_params = [{

    'labelling_method': LabellingChoice.price_move_in_window,
    'rolling_window': window,
    # Uncomment below if you want to check a price move only above a certain level
    'updown_threshold': threshold, #this is multiplied by 100
    'threshold_method': ThresholdMethod.arbitrary,
}]

for label_init in labelling_method_params:
    print label_init
    labeller = DataLabellingSimple(label_init)
    labeller.label_training_data(data)

keys_ = data.keys()

for key_, _ in enumerate(keys_):
    data[keys_[key_]].to_csv(ticker_labels_path+'/'+str(keys_[key_])+'.csv', index=False)


{'rolling_window': 25, 'labelling_method': <LabellingChoice.price_move_in_window: 'PrMov'>, 'updown_threshold': 0.1, 'threshold_method': <ThresholdMethod.arbitrary: 'arbitrary'>}


In [18]:
ticker_labels_path

'/home/ak/Data/features_models/labels/test_SYNT_2states/NON_DIRECTIONAL'

## For Labels: Locations we need ##

In [11]:
# ticker = 'SYNT_2states'
# ticker = 'SYNT_4states'


ticker_features_path = os.path.join(features_path, ticker)

###

# list of files    
labels_list = os.listdir(ticker_labels_path)

# features_list = os.listdir(ticker_features_path)

### Loading Labels###

In [12]:

non_directional = os.path.join(ticker_labels_path, labels_list[0])
labels_list

['NON_DIRECTIONAL', '20170605.csv', '20170602.csv', '20170601.csv']

In [27]:
data_dic = load_data(ticker, which_trading_hours=TradingHours.all_trading_day)
## clf fitting##
for date, date_hmm in trained_hmms.iteritems():
    feature_engine = hmm_features(date_hmm)
    features_load = feature_engine.generate_features(data_dic[date])
    labels_load= pd.read_csv(os.path.join(non_directional,str(date)+'.csv'))
    features, labels_clean = remove_nans(features_load, labels_load)
    x_std = sc.fit_transform(features.values.astype(np.float)) #fit & transform the features
    X_train, X_test, y_train, y_test = train_test_split( \
        x_std, labels_clean, test_size=0.05, random_state=1, stratify=labels_clean) #probably can get rid of this
    models_cls = FitModels(X_train, y_train)
    best_clfs = {#'SVC': models_cls.svm_clf(kernel_choice="rbf"),
                 'RIDGE_clf': models_cls.ridge_clf()
                 #'GBOOST': models_cls.gradient_boost_clf(),
                 'GP_clf': models_cls.gp_clf(),
                 #'RF_clf': models_cls.random_forest_clf(),
                 }
    # This is sequence for the name of the best classifiers.
    seq_clf = "_".join(("synthetic_data",str(date),labels_clean.columns.values[0],"clfs", ".pickle"))
    print("saving the classifiers:",seq_clf)
    pickle.dump(best_clfs, open(os.path.join(ticker_models_path,seq_clf), 'wb')) 

('saving the classifiers:', 'synthetic_data_20170601_label_PrMov__window_25__thres_arbitrary__10.0_clfs_.pickle')
('saving the classifiers:', 'synthetic_data_20170602_label_PrMov__window_25__thres_arbitrary__10.0_clfs_.pickle')
('saving the classifiers:', 'synthetic_data_20170605_label_PrMov__window_25__thres_arbitrary__10.0_clfs_.pickle')


In [28]:
best_clfs



{'RIDGE_clf': RidgeClassifierCV(alphas=array([1.000e-01, 2.000e-01, ..., 9.998e+02, 9.999e+02]),
          class_weight=None,
          cv=KFold(n_splits=5, random_state=None, shuffle=False),
          fit_intercept=True, normalize=True, scoring=None),
 'SVC': GridSearchCV(cv=None, error_score='raise',
        estimator=SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False),
        fit_params=None, iid=True, n_jobs=1,
        param_grid={'kernel': ['rbf'], 'C': [1, 5, 10, 25, 50, 100], 'gamma': [0.0001, 0.001, 0.01, 0.02, 0.05, 0.01]},
        pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
        scoring=None, verbose=0)}

In [ ]:
ticker_labels_path,

In [ ]:
pd.read_csv(os.path.join(ticker_labels_path,str(date)+'.csv'))